In [1]:
import numpy as np
import pandas as pd
import glob
import sys
import h5py

from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import pyarrow as pa
import pyarrow.parquet as pqw

from functools import reduce
import operator
import gc

h = 0.6774

In [2]:
# plot settings
plt.rc('font', family='serif') 
plt.rc('font', serif='Times New Roman') 
plt.rcParams.update({'font.size': 16})
plt.rcParams['mathtext.fontset'] = 'stix'

In [3]:
from pyspark import SparkContext   
from pyspark.sql import SparkSession

import pyspark.sql.functions as F
from pyspark.sql.functions import broadcast, col, sqrt, pow, floor, monotonically_increasing_id, abs, pmod, least, row_number
import pyspark.sql.types as T
from pyspark import Row
from pyspark.sql.window import Window as W

spark = SparkSession.builder \
    .appName("MyApp") \
    .master("spark://sohnic:7077") \
    .config("spark.driver.memory", "100g") \
    .getOrCreate()

sc = spark.sparkContext
sc.setCheckpointDir("hdfs://sohnic:54310/tmp/checkpoints")

spark.conf.set("spark.sql.debug.maxToStringFields", 500)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/02 16:55:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Reading the particle data

In [4]:
%%time

outname = 'hdfs://sohnic:54310/data/TNG300/snap99/parquet/snap099_sorted.parquet.snappy'
rawdf = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(outname)

24/09/02 16:55:35 WARN SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 4.51 ms, sys: 4.8 ms, total: 9.31 ms
Wall time: 9.56 s


In [5]:
rawdf.printSchema()

root
 |-- px: double (nullable = true)
 |-- py: double (nullable = true)
 |-- pz: double (nullable = true)
 |-- vx: double (nullable = true)
 |-- vy: double (nullable = true)
 |-- vz: double (nullable = true)
 |-- mass: double (nullable = true)



In [6]:
%%time
rawdf.limit(4).toPandas().T

CPU times: user 76.1 ms, sys: 5.12 ms, total: 81.2 ms
Wall time: 4.77 s


,0,1,2,3
px,56026.053658,56068.910690,55985.663601,56029.303588
py,89414.922492,89304.341870,89386.878305,89510.138206
pz,10320.077944,10294.866987,10308.034785,10500.129566
vx,-893.323200,-605.240700,-799.410800,-164.792340
vy,616.020750,585.237700,708.112900,31.142204
vz,374.236330,60.329155,569.718600,-605.351560
mass,0.000655,0.000463,0.000668,0.000623


In [7]:
%%time
rawdf.describe().show() 

+-------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+
|summary|                px|                py|                pz|                vx|                vy|                  vz|                mass|
+-------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+
|  count|         711967480|         711967480|         711967480|         711967480|         711967480|           711967480|           711967480|
|   mean|100688.61828005506| 99059.36895091066|105857.77590178423|1.2704478306706148|1.0880226031171318|-0.30910798969781916|5.280344892323576E-4|
| stddev| 58078.24131182934|57695.004162679215|59449.995547022576|376.97520515072165| 390.0023210124122|  379.19046823402147|1.424400845340055...|
|    min| 1.136348000727E-4|  8.96728655789E-4|   6.5989815512E-4|        -4127.3325|        -3954.2883|          -435

In [8]:
%%time
rawdf.describe().toPandas().T 

CPU times: user 165 ms, sys: 51 ms, total: 216 ms
Wall time: 44.7 s


,0,1,2,3,4
summary,count,mean,stddev,min,max
px,711967480,100688.61828005488,58078.24131182928,1.136348000727E-4,204999.9994621104
py,711967480,99059.36895091075,57695.0041626792,8.96728655789E-4,204999.9977547349
pz,711967480,105857.77590178432,59449.99554702274,6.5989815512E-4,204999.9998752371
vx,711967480,1.2704478306706108,376.9752051507225,-4127.3325,4329.7563
vy,711967480,1.0880226031171247,390.002321012412,-3954.2883,4393.859
vz,711967480,-0.309107989697819,379.19046823402107,-4355.7554,5640.448
mass,711967480,5.280344892323581E-4,1.4244008453400555E-4,8.154293E-6,0.0014874495


# 2. Calculating stellar velocity dispersion

We measure the stellar velocity dispersion of ~10$^6$ subhalos. We first select stellar particles within subhalo-centered spheres with a certain radius. We then calculate the mass-weighted stellar velocity dispersion based on the 3D velocity of selected stellar particles. 

### $M = \Sigma m_i $
### $\overline{v_x} = \frac{\Sigma m_i v_{i,x}}{M}$, $\, \overline{v_y} = \frac{\Sigma m_i v_{i,y}}{M}$, $\, \overline{v_z} = \frac{\Sigma m_i v_{i,z}}{M}$ 
### $\sigma_x^2 = \frac{\Sigma m_i (v_{i,x} - \overline{v_{x}})^2}{M}$, $\, \sigma_y^2 = \frac{\Sigma m_i (v_{i,y} - \overline{v_{y}})^2}{M}$, $\, \sigma_z^2 = \frac{\Sigma m_i (v_{i,z} - \overline{v_{z}})^2}{M}$
### $\therefore \sigma_{3D} = \sqrt{\sigma_x^2 + \sigma_y^2 + \sigma_z^2}$
We repeat the above calculation for six different radii (3, 5, 10, 20, 30, and 50 kpc).

In [14]:
%%time
#subhalo table
subname = 'hdfs://sohnic:54310/data/TNG300/snap99/parquet/subhalo_trim.parquet.snappy'
subdf = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(subname)

#adding a column for the subhalo id
subdf = subdf.withColumn("sub_id", monotonically_increasing_id())
windowSpec = W.orderBy("sub_id")
subdf = subdf.withColumn("sub_id", row_number().over(windowSpec) - 1)

#changing the column name for cross joining
subdf = subdf.withColumnRenamed("px", "sub_px")
subdf = subdf.withColumnRenamed("py", "sub_py")
subdf = subdf.withColumnRenamed("pz", "sub_pz")
subdf = subdf.withColumnRenamed("vx", "sub_vx")
subdf = subdf.withColumnRenamed("vy", "sub_vy")
subdf = subdf.withColumnRenamed("vz", "sub_vz")
subdf = subdf.withColumnRenamed("mass", "sub_mass")

subdf.describe().toPandas().T 

24/09/02 17:02:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/02 17:02:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/02 17:02:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/02 17:02:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


CPU times: user 19.8 ms, sys: 0 ns, total: 19.8 ms
Wall time: 607 ms


,0,1,2,3,4
summary,count,mean,stddev,min,max
sub_px,104290,99924.49163176889,57805.35894154692,1.0392832,204999.95
sub_py,104290,98057.49323886151,57423.51166697808,4.1745152,204999.47
sub_pz,104290,106564.75286649859,59506.67071144558,1.4059632,204998.62
sub_vx,104290,0.3373122921771032,409.5814311339034,-3211.1133,3810.9731
sub_vy,104290,1.2194316524949314,422.17403564986097,-3279.0154,3183.5188
sub_vz,104290,3.2695566500378765,411.9484211144669,-3364.2063,3228.1667
sub_mass,104290,3.899437017999686E10,1.0432081035999844E11,1.00000736E9,7.8380343E12
sub_id,104290,52144.5,30106.074124114333,0,104289


In [15]:
#dividing the data into 100*100*100 boxes and adding the column for the box id
subdf = subdf.withColumn("sub_ix", floor(col("sub_px") / 100))
subdf = subdf.withColumn("sub_iy", floor(col("sub_py") / 100))
subdf = subdf.withColumn("sub_iz", floor(col("sub_pz") / 100))

subdf.toPandas().head()

24/09/02 17:02:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/02 17:02:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/02 17:02:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/02 17:02:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/02 17:02:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,sub_px,sub_py,sub_pz,sub_vx,sub_vy,sub_vz,sub_mass,sub_id,sub_ix,sub_iy,sub_iz
0,99173.578125,40274.546875,107897.390625,-67.104362,-151.330200,167.907593,3.492153e+11,0,991,402,1078
1,120705.281250,69941.101562,157908.031250,269.752045,-320.683807,128.344238,3.287442e+11,1,1207,699,1579
2,204545.296875,170294.812500,87957.867188,363.489685,148.479889,-54.548824,3.453248e+11,2,2045,1702,879
3,173194.890625,158155.453125,162955.359375,83.774239,-126.974121,218.970139,3.937560e+11,3,1731,1581,1629
4,34154.796875,35974.179688,149377.250000,559.149902,597.246765,-471.297150,3.869574e+11,4,341,359,1493


In [16]:
%%time
#dividing the data into 100*100*100 boxes and adding the column for the box id
rawdf = rawdf.withColumn("ix", floor(col("px") / 100))
rawdf = rawdf.withColumn("iy", floor(col("py") / 100))
rawdf = rawdf.withColumn("iz", floor(col("pz") / 100))
rawdf.limit(4).toPandas()

CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 2.23 s


,px,py,pz,vx,vy,vz,mass,ix,iy,iz
0,101798.436134,130164.678844,19342.633655,-268.367740,-256.05673,-42.124146,0.000534,1017,1301,193
1,101767.527643,130187.124105,19372.751599,-291.788900,-472.26437,-258.926330,0.000314,1017,1301,193
2,101802.447651,130159.584576,19353.592309,-82.912390,-357.76523,-200.696210,0.000453,1018,1301,193
3,101760.288994,130185.236395,19383.717419,109.860855,-180.16005,-110.880310,0.000397,1017,1301,193


In [17]:
%%time

radius_sq = (50*h)**2 #50kpc aperture size
id_size = 100 #number of boxes
box_size = 205000 #box size in the ckpc/h unit

#broad casting the subhalo table to the all workers
broadcast_subdf = broadcast(subdf)

# Step 1: Coarse filtering by grid indices considering the periodic boundary
filtered_df = rawdf.crossJoin(broadcast_subdf).filter(
    (least(abs(col("ix") - col("sub_ix")), id_size - abs(col("ix") - col("sub_ix")) ) <= 1) & 
    (least(abs(col("iy") - col("sub_iy")), id_size - abs(col("iy") - col("sub_iy")) ) <= 1) & 
    (least(abs(col("iz") - col("sub_iz")), id_size - abs(col("iz") - col("sub_iz")) ) <= 1)
)

# Step 2: Fine filtering by distance calculation considering the periodic boundary
filtered_df = filtered_df.filter(
    (pow(least(abs(col("px") - col("sub_px")), box_size - abs(col("px") - col("sub_px"))), 2) +
     pow(least(abs(col("py") - col("sub_py")), box_size - abs(col("py") - col("sub_py"))), 2) +
     pow(least(abs(col("pz") - col("sub_pz")), box_size - abs(col("pz") - col("sub_pz"))), 2)) < radius_sq
)

# Select relevant columns (including subhalo_id for identification)
result_df = filtered_df.select("sub_id", "px", "py", "pz", "vx", "vy", "vz", "mass")

subname = 'hdfs://sohnic:54310/data/TNG300/snap99/parquet/extracted_region'
result_df.write.mode("overwrite").partitionBy("sub_id").parquet(subname)

# Checking the data

In [22]:
subname = 'hdfs://sohnic:54310/data/TNG300/snap99/parquet/extracted_region'
df = spark.read.parquet(subname)

df_filtered = df.filter(df.sub_id == 1)
df_filtered.describe().toPandas()

,summary,px,py,pz,vx,vy,vz,mass,sub_id
0,count,34296,34296,34296,34296,34296,34296,34296,34296
1,mean,120705.54486994068,69941.4977969158,157907.95237402615,269.3441459197633,-299.13841869982423,129.23422536839843,5.070395792973234E-4,1.0
2,stddev,10.132001551616199,8.894207648340112,5.512662270566082,217.87028374384283,204.64018849857712,149.82948770869788,1.358006157946126E-4,0.0
3,min,120671.60688671804,69907.39580384977,157875.3829258752,-570.1917,-1156.7537,-684.7255,3.146818E-5,1
4,max,120739.00671101324,69974.72385218015,157938.3532082483,1092.5302,498.63892,930.08295,9.543227E-4,1
